In [ ]:
import pandas as pd
print("Codespaces fonctionne !")

In [ ]:
import os
os.chdir('/workspaces/projet-iadata')  # Change le répertoire vers la racine
print(os.getcwd())  # Vérif
#qlq soucis avec les chemins


In [ ]:
#IMPORTER LES DATASET
conso= pd.read_csv('./data/brut/consommation-quotidienne-brute.csv')

print(conso.head()) #colums
print(conso.info()) #infos de la columns
print(conso.describe())
#constat :le fichier semble mal séparé (toute la ligne est import sur une meme colonne)

## NETTOYAGE DE DONNEES

### VERIFIER LES SEPARATEURS

In [ ]:
#use sep=";" pour mieux organisé le fichier
conso=pd.read_csv('./data/brut/consommation-quotidienne-brute.csv', sep=';')

#verif des colonnes
print(conso.head())
print(conso.info())
print(conso.describe())

##### RENOMMER LES COLONNNES

In [ ]:
# conso.columns['Date','Heure','Consommation', 'source','Unité']
# print(conso.head())
print(conso.columns)

In [ ]:
#certains noms sont trop long
# Renommer les colonnes clés pour simplifier leur manipulation
conso.rename(columns={
    'Date - Heure': 'DateHeure',
    'Date': 'Date',
    'Heure': 'Heure',
    'Consommation brute gaz totale (MW PCS 0°C)': 'ConsommationGazTotale',
    'Consommation brute électricité (MW) - RTE': 'ConsommationElectricite',
    'Consommation brute totale (MW)': 'ConsommationTotale',
    'Consommation brute gaz (MW PCS 0°C) - GRTgaz': 'ConsommationGazGRTgaz',
    'Consommation brute gaz (MW PCS 0°C) - Teréga': 'ConsommationGazTerega',
    'Statut - GRTgaz': 'StatutGRTgaz',
    'Statut - Teréga': 'StatutTerega',
    'Statut - RTE': 'StatutRTE'
}, inplace=True)


# Vérif des nouvelles colonnes
print(conso.columns)


In [ ]:
# Calculer le pourcentage de valeurs nulles par colonne
null_percentage = conso.isnull().mean() * 100

# Afficher les colonnes avec leurs pourcentages
print(null_percentage)

##### Gerer les valeurs maquantes

In [ ]:
# Calculer le pourcentage de valeurs nulles par colonne
null_percentage = conso.isnull().mean() * 100

# Afficher les colonnes avec leurs pourcentages
print(null_percentage)

In [ ]:
# uniquement les lignes contenant des valeurs NaN  dles colonnes importantes
conso = conso.dropna(subset=['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale'])
print(conso.info())  # Vérif que NaN ont été supprimés


##### colonnes inutiles

In [ ]:
#drop les colonnes inutiles
conso.drop(columns=['StatutGRTgaz', 'StatutTerega', 'StatutRTE'], inplace=True)
print(conso.info())

In [ ]:
#verif final

print(conso.isnull().sum())

##### DATA ABERRANTES

In [ ]:
# Recalculer la consommation totale en sommant le gaz et l'électricité
conso['ConsommationTotale'] = conso['ConsommationGazTotale'] + conso['ConsommationElectricite']

# Vérifiez les premières lignes pour confirmer le recalcul
print(conso[['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale']])


In [ ]:
#calcul de la médiane
# Calculer la médiane pour chaque colonne concernée
median_gaz = conso['ConsommationGazTotale'].median()
median_elec = conso['ConsommationElectricite'].median()

In [ ]:
# Moyenne et écart-type pour détect les aberrations
#mean : moyenne
#std : ecart-type 
mean_gaz = conso['ConsommationGazTotale'].mean()
std_gaz = conso['ConsommationGazTotale'].std() 

mean_elec = conso['ConsommationElectricite'].mean()
std_elec = conso['ConsommationElectricite'].std()


In [ ]:

# Remplacement des valeurs aberrantes par la médiane
conso.loc[
    (conso['ConsommationGazTotale'] > mean_gaz + 3 * std_gaz) | 
    (conso['ConsommationGazTotale'] < mean_gaz - 3 * std_gaz),
    'ConsommationGazTotale'
] = median_gaz

conso.loc[
    (conso['ConsommationElectricite'] > mean_elec + 3 * std_elec) | 
    (conso['ConsommationElectricite'] < mean_elec - 3 * std_elec),
    'ConsommationElectricite'
] = median_elec

In [ ]:
#A partir des nouvelles data
# Recalculer la consommation totale en sommant le gaz et l'électricité
conso['ConsommationTotale'] = conso['ConsommationGazTotale'] + conso['ConsommationElectricite']

# Vérifiez les premières lignes pour confirmer le recalcul
print(conso[['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale']])

SAUVEGARDE DES DATA CLEAN DANS UN NEW FILE

In [ ]:
# Vérification des modifications
print(conso.describe())

conso.to_csv('./data/prepared/conso_clean.csv', index=False) 